In [3]:
import numpy as np
FEATURES = np.load('./xgb_5fold/columns_names.npy')

In [4]:
len(FEATURES)

1658

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
data = pd.read_csv("train_logs.csv")
data['prev_cursor_position'] = data.groupby('id')['cursor_position'].shift(1).fillna(0)
corpus = []
student = []
def rebuild_text(student_id):

    #student.append(student_id)
    sub_data = data[data['id']==student_id].reset_index(drop=True)
    max_char = sub_data.cursor_position.max()
    chars = [''] * (max_char + 1)
    for row in sub_data.iterrows():
        row_idx, row_ele = row
        act_type = row_ele.activity
        cur_pos = row_ele.cursor_position
        prev_pos = int(row_ele.prev_cursor_position)
        text_change = row_ele.text_change
        if act_type == 'Input':
            chars[cur_pos-1] = text_change
        if act_type == 'Replace':
            before, after = text_change.split('=>')
            chars[prev_pos: prev_pos+len(before)] = [''] * len(before)
            chars[prev_pos: prev_pos+len(after)] = [i for i in after]
        if act_type == 'Remove/Cut':
            chars[prev_pos] = ''
          
    return ''.join(chars)


In [7]:
def getEssays(df):
    # Copy required columns
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change']].copy()
    
    # Get rid of text inputs that make no change
    # Note: Shift was unpreditcable so ignored
    textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']

    # Get how much each Id there is
    valCountsArr = textInputDf['id'].value_counts(sort=False).values
    # Holds the final index of the previous Id
    lastIndex = 0

    # Holds all the essays
    essaySeries = []

    # Fills essay series with essays
    for index, valCount in tqdm(enumerate(valCountsArr)):

        # Indexes down_time at current Id
        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change']].iloc[lastIndex : lastIndex + valCount]

        # Update the last index
        lastIndex += valCount

        # Where the essay content will be stored
        essayText = ""

        
        # Produces the essay
        for Input in currTextInput.values:
            
            # Input[0] = activity
            # Input[2] = cursor_position
            # Input[3] = text_change
            
            # If activity = Replace
            if Input[0] == 'Replace':
                # splits text_change at ' => '
                replaceTxt = Input[2].split(' => ')
                
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue

                
            # If activity = Paste    
            if Input[0] == 'Paste':
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue

                
            # If activity = Remove/Cut
            if Input[0] == 'Remove/Cut':
                # DONT TOUCH
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue

                
            # If activity = Move...
            if "M" in Input[0]:
                # Gets rid of the "Move from to" text
                croppedTxt = Input[0][10:]
                
                # Splits cropped text by ' To '
                splitTxt = croppedTxt.split(' To ')
                
                # Splits split text again by ', ' for each item
                valueArr = [item.split(', ') for item in splitTxt]
                
                # Move from [2, 4] To [5, 7] = (2, 4, 5, 7)
                moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))

                # Skip if someone manages to activiate this by moving to same place
                if moveData[0] != moveData[2]:
                    # Check if they move text forward in essay (they are different)
                    if moveData[0] < moveData[2]:
                        # DONT TOUCH
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        # DONT TOUCH
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue
                
                
            # If just input
            # DONT TOUCH
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]

            
        # Sets essay at index  
        
        essaySeries.append(essayText)
     
    
    # Sets essay series index to the ids

    #essaySeries.index =  textInputDf['id'].unique()
    
    
    # Returns the essay series
    return essaySeries

In [8]:
traindf = pd.read_csv('train_logs.csv')
text = getEssays(traindf)

qqqqqqqqq qq qqqqq qq qqqq qqqq.  qqqqqq qqq qqqq qqqqqq qq qq qqqqq qq qqqq qqqqq qq qqqqqqqqq qqqqq qqqq qqqqq qqq qqqqqqqqq qqqqqqqqq qqqq.  qqqqqq qqq qqqqq qqq qqqqqqqqqqq qq qqq qqqqqqqqqq qqqqq, qqq qqqqq qqqqqq qq qq qqqq qqq qqqqqq qqqqqqq qq qqq qqqqqqqqqqq.  qqqqqqqq qq qqqqqqqqqq qqqq qqqq qqqqqqqqq qqq qqqqqqq qq qqqqqq qqqq qqq qqq qq qqqqqqqqq qq qq qqq qqqqq qqqqq qq qqq.

qq qq qqqq qqqq qqq qqqqqqqqq qqq qqqqqqq qq qqq qqqqq qqqqq, qq qq qqqqqq qqq qqq qqqqqqqq qqqqq qq qqq qqqqqqqqqqq qq qqqqqqqqq.  qqqqqqqqq qq qqq qqqqqqqq qqqq qq qqqq qq qqqqqqq qqqqq qqqqq, qqq qqqqqq qqqqq qqqqq qqq qqq qq qqq qqqqqqq qqqqqqq qqqq.  qqqq qqqqq qqqqq qqqq qqqq'qq qqqqq qqqqqqqqq qqqqq qqqqqqq qqqqqqq qqqqqqqqqq, qqqq qq qqqqqqqqqq qqqqqqq qqq qqqqqqq; qqqqqqq, qqqqq qqqqqqqq qqqqqq qqqqqqq qqqqqqq qqq qqqqq qqq qqq qqq qqqqqqq.  qqqq qqqqqqqqq qqqq qqq qqqq qqqq qqqqq qqqqqqqqqq qqqq qqqqq qqqqq.  qqq qqqqqqqqqq qq qqqqqqqq q qqqqqq, qqqqqqqq qqqq qqqq qqqqqqqqqq, qqq. qq qqqqq q

/tmp/ipykernel_831741/2164982726.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  essaySeries = pd.Series()


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
from joblib import Parallel, delayed

results = Parallel(n_jobs=-1)(
    delayed(rebuild_text)(id_)
    for id_ in  data.id)

In [3]:
text = pd.DataFrame({'id': student,
                        'text': corpus
                      })

In [5]:
text

,id,text
0,001519c8,qqqqqqqqq qq qqqqq qq qqqq qqqq. q qq qqqq qq...
1,0022f953,"qqqq qq qqqqqqqqqqq ? qq qq qqq qqq qqq, qqqqq..."
2,0042269b,qqqqqqqqqqq qq qqqqq qqqqqqqqq qq qqqqqqqqqqq ...
3,0059420b,qq qqqqqqq qqqqqq qqqqqqqqqqqqq qqqq q qqqq qq...
4,0075873a,"qqqqqqqqqqq qq qqq qqqqq qq qqqqqqqqqq, qqq qq..."
...,...,...
2466,ffb8c745,"qq qqqqq'q qqqqqqq, qqq'q qqqqq q qqqq qq..."
2467,ffbef7e5,"qqqq qqqqqq qqqqq qq qqqqq qqqqq, qq qq q qqqq..."
2468,ffccd6fd,qqqqq qqqq q qqqqqqq qqqqqqqq qq qqqqqq qqqqqq...
2469,ffec5b38,"qqqqqqqqqq qqqqqqq, qqqqqq qqqq qqqqq qqqq qqq..."


In [4]:
text.to_csv('corpus.csv', index=None)

In [ ]:
ori = pd.read_csv('corpus.csv')